# AllenNLP tests

Use AllenNLP's semantic role labeling in combination with manually written rules for extracting causal relations from political speeches. We found two limitations: only relations within a single sentence were found and complex relations involving understanding the sentence were hard to identify.

Links:

* software installation: https://github.com/allenai/allennlp (do not forget to install NLTK popular models)
* software usage: https://demo.allennlp.org/semantic-role-labeling (tab: Model Usage)

## 1. Language processing modules

In [1]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
import spacy

We use AllenNLP for semantic role labeling and Spacy for lemmatization (of verbs).

In [2]:
spacy_analyze = spacy.load('en_core_web_sm')

In [3]:
srl_predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")

/home/erikt/anaconda3/envs/allennlp/lib/python3.7/site-packages/allennlp/tango/__init__.py:18: UserWarning: AllenNLP Tango is an experimental API and parts of it might change or disappear every time we release a new version.
  "AllenNLP Tango is an experimental API and parts of it might change or disappear "
2021-11-26 10:12:50,090 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2021-11-26 10:12:50,359 - INFO - cached_path - cache of https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz is up-to-date
2021-11-26 10:12:50,362 - INFO - allennlp.models.archival - loading archive file https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz from cache at /home/erikt/.allennlp/cache/b5f1db011cc85691a5fa2bf29e055a712261a2e5d74a74edd7da2fffc98d4ab8.4c4ac7e06ec3d85631bd26b839f90b5a375d3ceeb43e3c74f1cf4758dcee2bb3
2021-11-26 10:12:50,366 - INFO - allennlp.models.archival - extractin

## 2. Causal relation functions

In [67]:
import re

In [280]:
RED = 1
BLUE = 4
BLACK = 0
NO_COLOR = -1
COLOR_CONCEPT_1 = RED
COLOR_CONCEPT_2 = BLUE
COLOR_CONTENT_RELATION_EXPLANATION = BLACK
CONCEPT_1 = "Concept_1"
CONCEPT_2 = "Concept_2"
CONTENT_RELATION_EXPLANATION = "Content_Relation_Explanation"
NO_CONCEPT = ""
COLOR_CODES = { CONCEPT_1: COLOR_CONCEPT_1, CONCEPT_2: COLOR_CONCEPT_2, CONTENT_RELATION_EXPLANATION: COLOR_CONTENT_RELATION_EXPLANATION, NO_CONCEPT: NO_COLOR }
RELATION_VERBS = [ "achieve", "allow", "avoid", "balance", "boost", "bring", "brought", "consolidate", "compromise",  "contain", "contribute", 
                   "eliminate", "endanger", "entail", "ensure", "erode", "fail", "flow", "give", "increase", "open", "preserve", "protect", 
                   "reduce", "reinforce", "restore", "result", "safeguard", "secure", "strenghten", "support", "tackle", "trigger", "undermine", "weaken", ]
REVERSE_VERBS = [ "arise", "need" ]
ARGM_DIS_ROLES = [ "furthermore", "instead", "moreover", "nevertheless", "otherwise", "therefore", "thus" ]
NO_ARG0_ROLES = [ "he", "i", "she", "they", "we", "you" ]

In [5]:
def strip_tag(token):
    if len(token) > 1:
        if re.search("^\[", token):
            token = re.sub("^\[", "", token)
            token = re.sub(":$", "", token)
        token = re.sub("\]$", "", token)
    return token

In [6]:
def print_with_color(string, color_code):
    print(f"\x1b[1;3{color_code};47m{string}\x1b[m", end="")

In [187]:
def get_concept_tag_argm_prp(argument):
    if argument == "ARGM-PRP":
        return CONCEPT_2
    elif argument in ["V", "ARG0", "ARG1", "ARG2" ]:
        return CONCEPT_1
    return NO_CONCEPT

In [188]:
def get_concept_tag_argm_dis(argument):
    color_code = NO_COLOR
    if argument == "ARGM-DIS":
        return CONTENT_RELATION_EXPLANATION
    elif argument != "":
        return CONCEPT_2
    return NO_CONCEPT

In [189]:
def get_concept_tag_v(argument):
    if argument == "V":
        return CONTENT_RELATION_EXPLANATION
    elif argument in [ "ARG0", "ARG2", "ARGM-PRD" ]:
        return CONCEPT_1
    elif argument in [ "ARG1",  ]:
        return CONCEPT_2
    return NO_CONCEPT

In [190]:
def get_concept_tag_reverse_v(argument):
    if argument == "V":
        return CONTENT_RELATION_EXPLANATION
    elif argument in [ "ARG0", "ARG2", "ARGM-PRD" ]:
        return CONCEPT_2
    elif argument in [ "ARG1", ]:
        return CONCEPT_1
    return NO_CONCEPT

In [11]:
def check_verb(verb, RELATION_VERBS):
    if verb in RELATION_VERBS:
        return True
    if re.sub("[ds]$", "", verb) in RELATION_VERBS:
        return True
    if re.sub("ed$", "", verb) in RELATION_VERBS:
        return True
    if re.sub("ing$", "", verb) in RELATION_VERBS:
        return True
    if re.sub("ing$", "e", verb) in RELATION_VERBS:
        return True
    return False

In [80]:
def print_token(token, argument, first_token, last_token, color_code):
    if color_code == NO_COLOR:
        print_token_no_color(token, argument, first_token, last_token)
    else:
        print_token_color(token, argument, first_token, last_token, color_code)

In [81]:
def print_token_no_color(token, argument, first_token, last_token):
    if first_token:
        print(f"[{argument} ", end="")
    print(f"{token}", end="")
    if last_token:
        print(f"]", end="")
    print(" ", end="")

In [13]:
def print_token_color(token, argument, first_token, last_token, color_code):
    if first_token:
        print_with_color(f"[{argument} ", color_code)
    print_with_color(f"{token}", color_code)
    if last_token:
        print_with_color(f"]", color_code)
        print(" ", end="")
    else:
        print_with_color(" ", color_code)

In [27]:
def find_verb(tokens, arguments, lemmas):
    verb = ""
    if len(lemmas) != len(tokens):
        print_with_color(f"find_verb: error: different lengths for tokens ({len(tokens)}) and lemmas ({len(lemmas)})", 1)
    for i in range(0, len(arguments)):
        if arguments[i] == "V":
            verb = lemmas[i]
    return verb

In [98]:
def convert_analysis(analyzed_sentence):
    tokens_in = analyzed_sentence.split()
    arguments = []
    tokens_out = []
    first_tokens = []
    last_tokens = []
    current_argument = ""
    roles = {}
    first_token = False
    for i in range(0, len(tokens_in)):
        if re.search("^\[", tokens_in[i]):
            current_argument = strip_tag(tokens_in[i])
            first_token = True
            continue
        arguments.append(current_argument)
        tokens_out.append(strip_tag(tokens_in[i]))
        first_tokens.append(first_token)
        last_tokens.append(False)
        if current_argument != "":
            if current_argument not in roles:
                roles[current_argument] = strip_tag(tokens_in[i])
            else:
                roles[current_argument] += " " + strip_tag(tokens_in[i])
        if re.search("\]$", tokens_in[i]):
            current_argument = ""
            last_tokens[-1] = True
        first_token = False
    return [tokens_out, arguments, first_tokens, last_tokens, roles]

In [18]:
def get_lemmas(sentence):
    lemmas = []
    results = spacy_analyze(sentence)
    for token in results:
        lemmas.append(token.lemma_)
    return lemmas

In [191]:
def find_causal_relations(analyzed_sentence, lemmas):
    tokens, arguments, first_tokens, last_tokens, roles = convert_analysis(analyzed_sentence)
    concepts = []
    for i in range(0, len(tokens)):
        if "ARGM-PRP" in arguments:
            concepts.append(get_concept_tag_argm_prp(arguments[i]))
        elif "ARGM-DIS" in arguments and roles["ARGM-DIS"].lower() in ARGM_DIS_ROLES:
            concepts.append(get_concept_tag_argm_dis(arguments[i]))
        elif "V" in arguments and ("ARG0" not in roles or roles["ARG0"].lower() not in NO_ARG0_ROLES) and check_verb(find_verb(tokens, arguments, lemmas), RELATION_VERBS):
            concepts.append(get_concept_tag_v(arguments[i]))
        elif "V" in arguments and ("ARG0" not in roles or roles["ARG0"].lower() not in NO_ARG0_ROLES) and check_verb(find_verb(tokens, arguments, lemmas), REVERSE_VERBS):
            concepts.append(get_concept_tag_reverse_v(arguments[i]))
        else:
            concepts.append(NO_CONCEPT)
    return { "tokens": tokens, "arguments": arguments, "first_tokens": first_tokens, "last_tokens": last_tokens, "roles": roles, "concepts": concepts }

In [199]:
def pretty_print(causal_relation_data):
    for i in range(0, len(causal_relation_data["tokens"])):
        color_code = COLOR_CODES[causal_relation_data["concepts"][i]]
        print_token(causal_relation_data["tokens"][i], causal_relation_data["arguments"][i], causal_relation_data["first_tokens"][i], causal_relation_data["last_tokens"][i], color_code)
    print("")

In [208]:
def srl_analyze(sentence, filter="", print_flag=True):
    srl_analysis = srl_predictor.predict(sentence=sentence)
    causal_relations = []
    lemmas = get_lemmas(re.sub("\s+", " ", sentence))
    for verb_data in srl_analysis['verbs']:
        causal_relations.append(find_causal_relations(verb_data['description'], lemmas))
    if print_flag:
        for causal_relation_data in causal_relations:
            if filter == "" or re.search(filter, " ".join(causal_relation_data["tokens"] + causal_relation_data["arguments"])):
                pretty_print(causal_relation_data)
    return causal_relations

## 3. Test cases

In [197]:
srl_analyze("John sees the mountain with the snow")

[ARG0 John] [V sees] [ARG1 the mountain with the snow] 

In [200]:
srl_analyze("""You, in Greece, with our support, need to rebuild your country, your structures, your administration, 
               your economy to increase the competitiveness of Greece.""")

[ARG0 You] , [ARGM-LOC in Greece] , [ARGM-MNR with our support] , [V need] [ARG1 to rebuild your country , your structures , your administration , your economy to increase the competitiveness of Greece] . 
[ARG0 You] , in Greece , with our support , need to [V rebuild] [ARG1 your country , your structures , your administration , your economy] [ARGM-PRP to increase the competitiveness of Greece] . 
You , in Greece , with our support , need to rebuild your country , your structures , your administration , your economy to [V increase] [ARG1 the competitiveness of Greece] . 


In [36]:
srl_analyze("This opened the way to welfare gains from stronger economic and financial integration.")

[ARG0 This] [V opened] [ARG1 the way to welfare gains from stronger economic and financial integration] . 


In [37]:
srl_analyze("One risk is the temptation for governments to overborrow because the economic costs of excessive public debt")

[ARG1 One risk] [V is] [ARG2 the temptation for governments to overborrow because the economic costs of excessive public debt] 


In [38]:
srl_analyze("A resulting loosening of fiscal discipline in individual member states can endanger the stability-oriented monetary policy.")

A [V resulting] [ARG2 loosening] of fiscal discipline in individual member states can endanger the stability - oriented monetary policy . 
A resulting loosening of fiscal discipline in individual member states [V can] endanger the stability - oriented monetary policy . 
[ARG0 A resulting loosening of fiscal discipline in individual member states] [ARGM-MOD can] [V endanger] [ARG1 the stability - oriented monetary policy] . 
A resulting loosening of fiscal discipline in individual member states can endanger the [ARGM-MNR stability] - [V oriented] [ARG1 monetary policy] . 


In [41]:
srl_analyze("Therefore, the markets did not properly perform their expected policing function.")

Therefore , the markets [V did] not properly perform their expected policing function . 
[ARGM-DIS Therefore] , [ARG0 the markets] did [ARGM-NEG not] [ARGM-MNR properly] [V perform] [ARG1 their expected policing function] . 
Therefore , the markets did not properly perform their [V expected] [ARG1 policing function] . 


In [42]:
srl_analyze("I hope that following the lessons of interdependence not only at global, but also at European level given by the crisis")

[ARG0 I] [V hope] [ARG1 that following the lessons of interdependence not only at global , but also at European level given by the crisis] 
I hope that [V following] [ARG1 the lessons of interdependence] [ARGM-LOC not only at global , but also at European level given by the crisis] 
I hope that following [ARG1 the lessons of interdependence] not only at global , but also at European level [V given] [ARG0 by the crisis] 


In [43]:
srl_analyze("European economic strategy needs the full commitment of the European political community.")

[ARG0 European economic strategy] [V needs] [ARG1 the full commitment of the European political community] . 


In [44]:
srl_analyze("And a European economic system whose resilience flows from its single market")

And a European economic system [ARG1 whose resilience] [V flows] [ARGM-DIR from its single market] 


In [45]:
srl_analyze("there are of course considerable budgetary challenges arising from the recent exceptional measures")

there [V are] [ARGM-DIS of course] [ARG1 considerable budgetary challenges arising from the recent exceptional measures] 
there are of course [ARG1 considerable budgetary challenges] [V arising] [ARG2 from the recent exceptional measures] 


In [46]:
srl_analyze("And the best hope of a return to growth and job creation is inside the euro area.")

[ARGM-DIS And] [ARG1 the best hope of a return to growth and job creation] [V is] [ARG2 inside the euro area] . 


In [47]:
srl_analyze("To conclude, let me say a few words on the euro area more generally.")

To [V conclude] , let me say a few words on the euro area more generally . 
[ARGM-PRP To conclude] , [V let] [ARG1 me say a few words on the euro area more generally] . 
To conclude , let [ARG0 me] [V say] [ARG1 a few words] [ARGM-LOC on the euro area] [ARGM-ADV more generally] . 


In [48]:
srl_analyze("""We have taken important, fundamental decisions over the last couple of months to safeguard the stability 
               of the euro area, and indeed we are now in the phase of implementation.""")

We [V have] taken important , fundamental decisions over the last couple of months to safeguard the stability of the euro area , and indeed we are now in the phase of implementation . 
[ARG0 We] have [V taken] [ARG1 important , fundamental decisions] [ARGM-TMP over the last couple of months] [ARGM-PRP to safeguard the stability of the euro area] , and indeed we are now in the phase of implementation . 
[ARG0 We] have taken important , fundamental decisions over the last couple of months to [V safeguard] [ARG1 the stability of the euro area] , and indeed we are now in the phase of implementation . 
We have taken important , fundamental decisions over the last couple of months to safeguard the stability of the euro area , and [ARGM-ADV indeed] [ARG1 we] [V are] [ARGM-TMP now] [ARG2 in the phase of implementation] . 


In [49]:
srl_analyze("""A number of governments have embarked on a path of reform and fiscal consolidation that was unthinkable only 
               very recently, and they have taken important decisions and I encourage them to keep this determination.""")

A number of governments [V have] embarked on a path of reform and fiscal consolidation that was unthinkable only very recently , and they have taken important decisions and I encourage them to keep this determination . 
[ARG0 A number of governments] have [V embarked] [ARG1 on a path of reform and fiscal consolidation that was unthinkable only very recently] , and they have taken important decisions and I encourage them to keep this determination . 
A number of governments have embarked on [ARG1 a path of reform and fiscal consolidation] [R-ARG1 that] [V was] [ARG2 unthinkable] [ARGM-TMP only very recently] , and they have taken important decisions and I encourage them to keep this determination . 
A number of governments have embarked on a path of reform and fiscal consolidation that was unthinkable only very recently , and they [V have] taken important decisions and I encourage them to keep this determination . 
A number of governments have embarked on a path of reform and fiscal con

In [50]:
srl_analyze("""These reforms are now being implemented and this effort must continue with credibility, with consistency, 
               with coherence over time.""")

These reforms [V are] now being implemented and this effort must continue with credibility , with consistency , with coherence over time . 
These reforms are now [V being] implemented and this effort must continue with credibility , with consistency , with coherence over time . 
[ARG1 These reforms] are [ARGM-TMP now] being [V implemented] and this effort must continue with credibility , with consistency , with coherence over time . 
These reforms are now being implemented and this effort [V must] continue with credibility , with consistency , with coherence over time . 
These reforms are now being implemented and [ARG1 this effort] [ARGM-MOD must] [V continue] [ARGM-MNR with credibility , with consistency , with coherence over time] . 


In [51]:
srl_analyze("As we said there will not be magic solutions.")

As [ARG0 we] [V said] there will not be magic solutions . 
As we said there [V will] not be magic solutions . 
[ARGM-ADV As we said] there [ARGM-MOD will] [ARGM-NEG not] [V be] [ARG1 magic solutions] . 


In [52]:
srl_analyze("We need sustained efforts and determination.")

[ARG0 We] [V need] [ARG1 sustained efforts and determination] . 


In [53]:
srl_analyze("At the same time, the existing financial backstops are being used as necessary.")

At the same time , the [V existing] [ARG1 financial backstops] are being used as necessary . 
At the same time , the existing financial backstops [V are] being used as necessary . 
At the same time , the existing financial backstops are [V being] used as necessary . 
[ARGM-TMP At the same time] , [ARG1 the existing financial backstops] are being [V used] [ARGM-MNR as necessary] . 


In [54]:
srl_analyze("""Most recently, the financial assistance to the recapitalisation of Spanish banks has been agreed 
               and is ready for implementation.""")

Most recently , the financial assistance to the recapitalisation of Spanish banks [V has] been agreed and is ready for implementation . 
Most recently , the financial assistance to the recapitalisation of Spanish banks has [V been] agreed and is ready for implementation . 
[ARGM-TMP Most recently] , [ARG1 the financial assistance to the recapitalisation of Spanish banks] has been [V agreed] and is ready for implementation . 
[ARGM-TMP Most recently] , [ARG1 the financial assistance to the recapitalisation of Spanish banks] has been agreed and [V is] [ARG2 ready for implementation] . 


In [55]:
srl_analyze("""Giving to the ECB the ultimate responsibility for supervision of banks in the euro area 
               will decisively contribute to increase confidence between the banks 
               and in this way increase the financial stability in the euro area.""")

[V Giving] [ARG2 to the ECB] [ARG1 the ultimate responsibility for supervision of banks in the euro area] will decisively contribute to increase confidence between the banks and in this way increase the financial stability in the euro area . 
Giving to the ECB the ultimate responsibility for supervision of banks in the euro area [V will] decisively contribute to increase confidence between the banks and in this way increase the financial stability in the euro area . 
[ARGM-PRD Giving to the ECB the ultimate responsibility for supervision of banks in the euro area] [ARGM-MOD will] [ARGM-MNR decisively] [V contribute] [ARG2 to increase confidence between the banks and in this way increase the financial stability in the euro area] . 
Giving to the ECB the ultimate responsibility for supervision of banks in the euro area will decisively contribute to [V increase] [ARG1 confidence between the banks] and in this way increase the financial stability in the euro area . 
Giving to the ECB the u

In [56]:
srl_analyze("Second, Germany is acutely aware of the need to tackle the root causes and not just the symptoms of the crisis.")

[ARGM-DIS Second] , [ARG1 Germany] [V is] [ARG2 acutely aware of the need to tackle the root causes and not just the symptoms of the crisis] . 
Second , Germany is acutely aware of the need to [V tackle] [ARG1 the root causes and not just the symptoms of the crisis] . 


In [57]:
srl_analyze("""This is why it is pressing strongly for institutional reforms of the EMU framework plus structural reforms 
               and budgetary discipline in the member states.""")

[ARG1 This] [V is] [ARG2 why it is pressing strongly for institutional reforms of the EMU framework plus structural reforms and budgetary discipline in the member states] . 
This is why it [V is] pressing strongly for institutional reforms of the EMU framework plus structural reforms and budgetary discipline in the member states . 
This is [ARGM-CAU why] [ARG0 it] is [V pressing] [ARGM-MNR strongly] [ARG2 for institutional reforms of the EMU framework plus structural reforms and budgetary discipline in the member states] . 


In [58]:
srl_analyze("The introduction of the euro eliminated exchange rate risks.")

[ARG0 The introduction of the euro] [V eliminated] [ARG1 exchange rate risks] . 


In [59]:
srl_analyze("""Another implication of the euro area's single monetary policy is that the key interest rates 
               are set for the currency bloc as a whole.""")

[ARG1 Another implication of the euro area 's single monetary policy] [V is] [ARG2 that the key interest rates are set for the currency bloc as a whole] . 
Another implication of the euro area 's single monetary policy is that the key interest rates [V are] set for the currency bloc as a whole . 
Another implication of the euro area 's single monetary policy is that [ARG1 the key interest rates] are [V set] [ARGM-ADV for the currency bloc as a whole] . 


In [60]:
srl_analyze("""One risk is the temptation for governments to overborrow because the economic costs of excessive public debt, 
               for example higher interest rates, can be more easily shifted to other member states.""")

[ARG1 One risk] [V is] [ARG2 the temptation for governments to overborrow because the economic costs of excessive public debt , for example higher interest rates , can be more easily shifted to other member states] . 
One risk is the temptation for governments to overborrow because the economic costs of excessive public debt , for example higher interest rates , [V can] be more easily shifted to other member states . 
One risk is the temptation for governments to overborrow because the economic costs of excessive public debt , for example higher interest rates , can [V be] more easily shifted to other member states . 
One risk is the temptation for governments to overborrow because [ARG1 the economic costs of excessive public debt , for example higher interest rates] , [ARGM-MOD can] be [ARGM-MNR more easily] [V shifted] [ARG2 to other member states] . 


In [61]:
srl_analyze("""Even Germany ran up excessive deficits for a few years and, even worse, championed a reform of the SGP 
               which ultimately further weakened the application of the fiscal rules.""")

[ARG0 Even Germany] [V ran] up [ARG1 excessive deficits] [ARGM-TMP for a few years] and , even worse , championed a reform of the SGP which ultimately further weakened the application of the fiscal rules . 
[ARG0 Even Germany] ran up excessive deficits for a few years and , [ARGM-ADV even worse] , [V championed] [ARG1 a reform of the SGP which ultimately further weakened the application of the fiscal rules] . 
Even Germany ran up excessive deficits for a few years and , even worse , championed [ARG0 a reform of the SGP] [R-ARG0 which] [ARGM-TMP ultimately] [ARG2 further] [V weakened] [ARG1 the application of the fiscal rules] . 


In [62]:
srl_analyze("""However, the EMU framework not only failed to avoid excessive deficits, it was also unable to prevent 
               the build-up of macroeconomic imbalances within the euro area.""")

[ARGM-DIS However] , [ARG1 the EMU framework] not only [V failed] [ARG2 to avoid excessive deficits] , it was also unable to prevent the build - up of macroeconomic imbalances within the euro area . 
However , [ARG0 the EMU framework] not only failed to [V avoid] [ARG1 excessive deficits] , it was also unable to prevent the build - up of macroeconomic imbalances within the euro area . 
However , the EMU framework not only failed to avoid excessive deficits , [ARG1 it] [V was] [ARGM-ADV also] [ARG2 unable to prevent the build - up of macroeconomic imbalances within the euro area] . 
However , the EMU framework not only failed to avoid excessive deficits , [ARG0 it] was also unable to [V prevent] [ARG1 the build - up of macroeconomic imbalances within the euro area] . 


In [63]:
srl_analyze("""The resulting increase in domestic inflation and wages eroded the competitiveness of the countries concerned and 
               increased their dependence on capital imports.""")

The [V resulting] [ARG2 increase] in domestic inflation and wages eroded the competitiveness of the countries concerned and increased their dependence on capital imports . 
[ARG0 The resulting increase in domestic inflation and wages] [V eroded] [ARG1 the competitiveness of the countries concerned] and increased their dependence on capital imports . 
The resulting increase in domestic inflation and wages eroded the competitiveness of [ARG1 the countries] [V concerned] and increased their dependence on capital imports . 
[ARG0 The resulting increase in domestic inflation and wages] eroded the competitiveness of the countries concerned and [V increased] [ARG1 their dependence on capital imports] . 


In [64]:
srl_analyze("The task of implementing the reforms and regaining competitiveness entailed significant political and social costs.")

The task of [V implementing] [ARG1 the reforms] and regaining competitiveness entailed significant political and social costs . 
The task of implementing the reforms and [V regaining] [ARG1 competitiveness] entailed significant political and social costs . 
[ARG0 The task of implementing the reforms and regaining competitiveness] [V entailed] [ARG1 significant political and social costs] . 


In [65]:
srl_analyze("""However, these efforts, supported by a strong expansion in the global economy, allowed German growth to rebound 
               after 2005.""")

However , [ARG1 these efforts] , [V supported] [ARG0 by a strong expansion in the global economy] , allowed German growth to rebound after 2005 . 
[ARGM-DIS However] , [ARG0 these efforts , supported by a strong expansion in the global economy] , [V allowed] [ARG1 German growth to rebound after 2005] . 
However , these efforts , supported by a strong expansion in the global economy , allowed [ARG1 German growth] to [V rebound] [ARGM-TMP after 2005] . 


In [66]:
srl_analyze("""In order to achieve a turnaround and allow further assistance, it is now essential for Greece to deliver 
               on the promises that have been made.""")

In order to [V achieve] [ARG1 a turnaround] and allow further assistance , it is now essential for Greece to deliver on the promises that have been made . 
In order to achieve a turnaround and [V allow] [ARG1 further assistance] , it is now essential for Greece to deliver on the promises that have been made . 
[ARGM-PRP In order to achieve a turnaround and allow further assistance] , it [V is] [ARGM-TMP now] [ARG2 essential] [ARG1 for Greece to deliver on the promises that have been made] . 
In order to achieve a turnaround and allow further assistance , it is now essential for [ARG0 Greece] to [V deliver] [ARG1 on the promises that have been made] . 
In order to achieve a turnaround and allow further assistance , it is now essential for Greece to deliver on the promises that [V have] been made . 
In order to achieve a turnaround and allow further assistance , it is now essential for Greece to deliver on the promises that have [V been] made . 
In order to achieve a turnaround and allow

## 4. Propbank role explanation

Modifiers in Propbank (source: http://clear.colorado.edu/compsem/documents/propbank_guidelines.pdf )
* ADJ: Adjectival
* ADV: Adverbials
* CAU: Cause
* COM: Comitative
* DIR: Directional
* DIS: Discourse
* DSP: Direct Speech
* EXT: Extent
* GOL: Goal
* LOC: Locative
* LVB: Light Verb
* MNR: Manner
* MOD: Modal
* NEG: Negation
* PRD: Secondary Predication
* PRP: Purpose
* REC: Reciprocals
* SLC: Relative Clause

## 5. Process texts

In [68]:
import json

In [69]:
def read_data(file_name):
    json_data = []
    infile = open(file_name, "r")
    for line in infile:
        json_data.append(json.loads(line))
    infile.close()
    return json_data

In [245]:
def get_concepts_json(json_data_element):
    concepts = {}
    for label_element in json_data_element["label"]:
        label = label_element[2]
        phrase = json_data_element["data"][label_element[0]: label_element[1]]
        if label not in concepts:
            concepts[label] = phrase
        else:
            concepts[label] += " " + phrase
    return list(concepts.items())

In [246]:
def get_concepts_srl(srl_data):
    concepts = {}
    for i in range(0, len(srl_data["concepts"])):
        if srl_data["concepts"][i] != NO_CONCEPT:
            label = srl_data["concepts"][i]
            phrase = srl_data["tokens"][i]
            if label not in concepts:
                concepts[label] = phrase
            elif len(phrase) > 1 or re.search("\w", phrase):
                concepts[label] += " " + phrase
            else:
                concepts[label] += phrase
    return list(concepts.items())

In [262]:
def evaluate_concepts(concepts_json, concepts_srl):
    nbr_of_shared = 0
    concepts_srl_used = len(concepts_srl) * [False]
    for i in range(0, len(concepts_json)):
        for j in range(0, len(concepts_srl)):
            if not concepts_srl_used[j] and concepts_srl[j][0] == concepts_json[i][0] and concepts_srl[j][1] == concepts_json[i][1]:
                nbr_of_shared += 1
                concepts_srl_used[j] = True
                break
    print(f"precision: {nbr_of_shared/len(concepts_srl):.3f}; recall: {nbr_of_shared/len(concepts_json):.3f}")

In [70]:
json_data = read_data("../data/femke-20211012.jsonl")

In [273]:
get_concepts_json(json_data[20])

[('Content_Concept_1', 'introduction of the euro'),
 ('Content_Relation_Explanation', 'eliminated'),
 ('Content_Concept_2', 'exchange rate risks')]

In [281]:
seen = {}
for paragraph_id in range(20, 21):
    paragraph_text = json_data[paragraph_id]["data"]
    if paragraph_text not in seen:
        print("####################")
        concepts_json = get_concepts_json(json_data[paragraph_id])
        concepts_srl = []
        spacy_analysis = spacy_analyze(paragraph_text)
        for sentence in spacy_analysis.sents:
            srl_analysis = srl_analyze(str(sentence), filter="")
            for srl_analysis_data in srl_analysis:
                concepts_srl.extend(get_concepts_srl(srl_analysis_data))
            print("")
        seen[paragraph_text] = True
        print(f"{paragraph_id}: ", end="")
        evaluate_concepts(concepts_json, concepts_srl)

####################
[ARG0 The introduction of the euro] [V eliminated] [ARG1 exchange rate risks] . 

[ARG0 This] [V opened] [ARG1 the way to welfare gains from stronger economic and financial integration] . 

[ARG0 You] [V pay] [ARGM-MNR with the same currency] , [ARGM-ADV no matter whether you are in Dublin , Helsinki , Lisbon or Rome] . 
You pay with the same currency , no matter whether [ARG1 you] [V are] [ARG2 in Dublin , Helsinki , Lisbon or Rome] . 

More importantly , the euro [V has] proved to be a very stable currency - with a strong exchange rate and an annual inflation rate of about 2 % since its introduction . 
[ARGM-ADV More importantly] , [ARG1 the euro] has [V proved] [C-ARG1 to be a very stable currency - with a strong exchange rate and an annual inflation rate of about 2 % since its introduction] . 
More importantly , [ARG1 the euro] has proved to [V be] [ARG2 a very stable currency - with a strong exchange rate and an annual inflation rate of about 2 % since its int